# CatBoost Algorithm

CatBoost is a state-of-the-art open-source gradient boosting on decision trees library. It's simple and easy to use. And is now regularly one
of the top algorithms used in data science competitions as it produces very good results without extensive data clean—up or feature
engineering.

In [ ]:
# !pip install catboost -q


In [5]:
# imports libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from catboost import CatBoostClassifier

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# load the titanic dataset
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [8]:
#impute missing values of age, fare, embarked, embark_town using KNN imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[['age', 'fare']] = imputer.fit_transform(df[['age', 'fare']])
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
df['embark_town'].fillna(df['embark_town'].mode()[0], inplace=True)

# drop the deck column
df.drop('deck', axis=1, inplace=True)


C:\Users\Sumit Sharma\AppData\Local\Temp\ipykernel_16208\2151508802.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
C:\Users\Sumit Sharma\AppData\Local\Temp\ipykernel_16208\2151508802.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [ ]:
# convert the category columns to category
cat_cols = df.select_dtypes(include=['object','category']).columns,

df[cat_cols] = df[cat_cols].astype('category')

In [ ]:
# split the data into Features X and Target y
X = df.drop('survived',axis=1)
y = df["survived"]

# trian and test the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create the model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1,
                           depth=6,
                           loss_function='Logloss',
                           eval_metric='Accuracy',
                           random_seed=42,
                           verbose=False)

# fit the model
model.fit(X_train, y_train, cat_features=cat_cols.tolist())

# predict
y_pred = model.predict(X_test)

# evalute the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
print('Classification: \n', classification_report(y_test, y_pred))



In [10]:
df.isnull().sum().sort_values(ascending=False)

deck           688
pclass           0
survived         0
age              0
sibsp            0
parch            0
sex              0
fare             0
embarked         0
who              0
class            0
adult_male       0
embark_town      0
alive            0
alone            0
dtype: int64